In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import glob

In [ ]:
vocab = ['electrons','protons','muons','pions','gamma']

image_size = 128

minibatch_size = 256 

epochs = 1000
learning_rate=0.0001
training_size = 1500

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components and one hot them
    parts = tf.string_split([file_path], '/')
    label = parts.values[-2] 
    matches = tf.stack([tf.equal(label, s) for s in vocab], axis=-1)
    onehot = tf.cast(matches, tf.float32)
    return onehot

In [ ]:
def get_label_csv(file_path):
    parts = tf.string_split([file_path], '/')
    label = parts.values[-3] 
    matches = tf.stack([tf.equal(label, s) for s in vocab], axis=-1)
    onehot = tf.cast(matches, tf.float32)
    return onehot

In [ ]:
def decode_img(img):
  # convert the compressed string to a uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  img = tf.image.per_image_standardization(img)
  return tf.image.resize(img, [image_size, image_size])

In [ ]:
def process_image(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
def decode_csv(csv):
#     rDefaults = [[0.]] * 8 + [[0]] 
#     csv= tf.io.decode_csv(csv, record_defaults=rDefaults)
    parsed_line = tf.decode_csv(csv, [[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]])
    return parsed_line
    

In [ ]:
def process_csv(file_path):
    #csv = tf.data.experimental.make_csv_dataset(file_path,batch_size=128)
    #csv = tf.data.experimental.CsvDataset(file_path,header=True,record_defaults=[tf.f]*8)
    
#     csv = tf.io.read_file(file_path)
    
    label = get_label_csv(file_path)
    dataset = decode_csv(file_path)
#     csv = decode_csv(csv)
#     csv = pd.read_csv(file_path)
#     dataset = tf.data.Dataset.from_tensor_slices((df.values))
    
    
    return dataset,label

    
    

In [1]:
def load_dataset(image_dir,frequency_dir,minibatch_size,num_parallel_calls=8):
    list_ds = tf.data.Dataset.list_files(image_dir)
    
#     list_frequency_ds = tf.data.Dataset.list_files(frequency_dir)
# # Set `num_parallel_calls` so multiple images are loaded/processed in parallel.

    image_dataset = list_ds.map(process_image, num_parallel_calls)  
    batched_image_dataset = image_dataset.batch(minibatch_size).prefetch(1)
    

   # csv_dataset = tf.data.TextLineDataset(frequency_dir).skip(1).map(decode_csv)

    csv_dataset = tf.data.experimental.make_csv_dataset(frequency_dir,batch_size=8,shuffle=False)
    
    dataset = [batched_image_dataset,csv_dataset]
    return dataset